# Overview

The Maternal Artificial Intelligence project demonstrates a simple approach to utilize website content as a context for answering questions related to the website's subject. In this example, we use a website called "Sou Mãe," which has over 2,000 blog posts. These posts are converted into a CSV format, and various techniques are applied to use this content as a context for answering questions.

# Key Concepts
* **Loading Data**: The data is loaded from a CSV file containing blog posts.
* **Preprocessing Data**: The blog posts are cleaned by removing HTML tags, newlines, and other unnecessary characters.
* **Embeddings**: Embeddings are used to represent the blog posts in a continuous vector space. OpenAI's API is utilized to create embeddings for each post.
* **Similarity**: The similarity between a query and the embeddings is calculated using the dot product, which represents the cosine similarity in this case.
* **Context Selection**: The most relevant sections of the blog posts are selected based on their similarity to the query.
* **Question Answering**: OpenAI's Completion API is used to generate answers to queries, using the selected contexts.

# Code Breakdown

## Importing Libraries

The code starts by importing necessary libraries, such as numpy, openai, pandas, re, csv, and time. These libraries will be used throughout the code to process data and interact with the OpenAI API.

In [53]:
import numpy as np
import openai
import pandas as pd
import pickle
import tiktoken
import re
import csv
import time
from typing import Dict, List, Tuple

## Configuration

Here, we define some essential configuration values, such as the OpenAI API key, model names, maximum section length, separator, and encoding.

In [69]:
# Configuration
openai.api_key = "PUT_YOUR_KEY_HERE"
COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDING_MODEL = "text-embedding-ada-002"

## Loading and Preprocessing Data
The code reads data from a CSV file named posts_complete.csv and preprocesses it by removing HTML tags, newlines, carriage returns, tabs, and square brackets.

In [54]:
# Load data
df = pd.read_csv('posts_complete.csv')

# Preprocess data
df['post_content'] = df.post_content.apply(lambda x: re.sub('<[^<]+?>', '', str(x)))
df['post_content'] = df.post_content.apply(lambda x: re.sub('\n', '', str(x)))
df['post_content'] = df.post_content.apply(lambda x: re.sub('\r', '', str(x)))
df['post_content'] = df.post_content.apply(lambda x: re.sub('\t', '', str(x)))
df['post_content'] = df.post_content.apply(lambda x: re.sub('\[.*\]', '', str(x)))

#df = df.set_index(["post_title", "post_content"])
print(f"{len(df)} rows in the data.")
df.sample(5)

2236 rows in the data.


,post_title,post_content
1770,Teste de gravidez online - será que funciona? ...,Como funciona um teste de gravidez online? Em ...
1710,Dicas para dirigir com segurança com crianças ...,"Paraevitar acidentes, selecionamos algumas dic..."
185,Decoração de banheiro infantil,Algumas casas tem um banheiro a mais para as c...
467,Como fazer a limpeza do coto umbilical?,Os pais de primeira viagem costumam ficar nerv...
949,Vídeo - Recebidos e Comprinhas de Maio,"Olá mamães, vídeo novo no ar!Confiram os produ..."


## Functions
Several functions are defined to handle different aspects of the code:

* `get_embedding()`: Retrieves embeddings from OpenAI API for the given text.
* `compute_doc_embeddings()`: Creates document embeddings for each row in the DataFrame and saves them to a CSV file.
* `load_embeddings()`: Loads document embeddings from a CSV file.
* `vector_similarity()`: Calculates the similarity between two vectors.
* `order_document_sections_by_query_similarity()`: Orders document sections based on their similarity to a given query.
* `construct_prompt()`: Constructs a prompt for the OpenAI API based on the most relevant document sections.
* `answer_query_with_context()`: Sends the constructed prompt to the OpenAI API and receives the answer to the query.

## Loading or Computing Embeddings
The code allows you to either compute the embeddings from scratch using the `compute_doc_embeddings()` function or load them from a CSV file using the `load_embeddings()` function.

In [55]:
def get_embedding(text: str, model: str = EMBEDDING_MODEL) -> List[float]:
    delay = 1
    time.sleep(delay)
    
    result = openai.Embedding.create(
        model=model,
        input=text
    )
    return result["data"][0]["embedding"]

def compute_doc_embeddings(df: pd.DataFrame) -> Dict[Tuple[str, str], List[float]]:
    """
    Create an embedding for each row in the dataframe using the OpenAI Embeddings API.

    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    # here write the return in a file
    file = open('embeddings.csv', 'w')
    writer = csv.writer(file)

    embeddings_dict = {}

    for idx, r in df.iterrows():
        post_content = r['post_content']
        embedding = get_embedding(post_content)
        key = (idx, post_content)
        embeddings_dict[key] = embedding
        # grava em arquivo linha a linha
        list = []
        list.append(key[0])
        list.append(key[1])
        for k in embedding:
            list.append(k)
        writer.writerow(list)
        print(key[0])

    return embeddings_dict

In [56]:
def load_embeddings(fname: str) -> Dict[Tuple[str, str], List[float]]:
    """
    Read the document embeddings and their keys from a CSV.
    
    fname is the path to a CSV with exactly these named columns: 
        "title", "heading", "0", "1", ... up to the length of the embedding vectors.
    """
    
    df = pd.read_csv(fname, header=0)
    max_dim = max([int(c) for c in df.columns if c != "title" and c != "heading"])
    return {
           (r.title, r.heading): [r[str(i)] for i in range(max_dim + 1)] for _, r in df.iterrows()
    }

In [57]:
#Here you can load_embeddings from a CSV or compute all embeddings again

#document_embeddings = compute_doc_embeddings(df)
document_embeddings = load_embeddings("embeddings.csv")

In [58]:
# An example embedding:
example_entry = list(document_embeddings.items())[0]
print(f"{example_entry[0]} : {example_entry[1][:5]}... ({len(example_entry[1])} entries)")

(0, 'Como\xa0falar com o SouMãe.org?Se você chegou aqui nessa página é porque deseja entrar em contato com SouMãe.org. Se você tem alguma dúvida ou sugestão mande um email para gisele@soumae.org ou preencha o formulário a baixo.Você é empresa e acha que seu produto combina com o nosso blog, deseja saber como\xa0anunciar aqui no\xa0SouMãe.org? Entre em contato para verificar as opções de anunciar\xa0no SouMãe.org.') : [0.0035688076168298, -0.0009585248772054, -0.0086283031851053, -0.0093294316902756, -0.0219939611852169]... (1536 entries)


# Similarity and Context Selection Functions: 
Define functions for calculating similarity between embeddings and selecting relevant contexts.

In [59]:
def vector_similarity(x: List[float], y: List[float]) -> float:
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

def order_document_sections_by_query_similarity(query: str, contexts: Dict[Tuple[str, str], np.ndarray]) -> List[Tuple[float, Tuple[str, str]]]:
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities


In [60]:
order_document_sections_by_query_similarity("Quem é a Gisele?", document_embeddings)[:2]


[(0.8437692218247463,
  (429,
   'Então… para quem não sabe, sou Gisele, 31 anos, casada, com uma filha que vai fazer 2 anos daqui a alguns dias. Também sou dona aqui do\xa0soumae.org, esse\xa0site existe desde 12 de setembro de 2014, onde o foco é dar dicas de decoração de festas infantis, chá de bebê, decoração de quartos e dicas para gestantes e mães… mas ele não é um blog pessoal… ele é meu trabalho. Gosto muito, lógico! Mas é trabalho!Eu não trabalho fora, decidimos, marido e eu, que eu ficaria em casa cuidando da nossa filhota, Isabela, e agradeço a Deus por ter essa possibilidade de passar esse tempo precioso com ela! Com isso acabei criando o\xa0soumae.org\xa0que hoje tem mais de 700 mil visualizações por mês! IURRU!Voltando ao assunto “blog novo”… As vezes eu até tento escrever algo um pouco mais pessoal no soumae…\xa0como dicas de atividades que faço com minha filha, ou outros assuntos (bem raro), mas esse não é o foco do soumae, até que por fim desisti de compartilhar coisas

# Answering Queries: 
Define a function that utilizes OpenAI's Completion API to generate answers to queries using the selected contexts.

In [61]:
MAX_SECTION_LEN = 2000
SEPARATOR = "\n* "
ENCODING = "gpt2"  # encoding for text-davinci-003

encoding = tiktoken.get_encoding(ENCODING)
separator_len = len(encoding.encode(SEPARATOR))

f"Context separator contains {separator_len} tokens"

'Context separator contains 3 tokens'

In [62]:
def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame) -> str:
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
    
    for _, section_index in most_relevant_document_sections:
        # Add contexts until we run out of space.        
        document_section = df.loc[section_index[0]]
        chosen_sections_len += len(encoding.encode(document_section[1])) + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + document_section[1].replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
#     print(f"Selected {len(chosen_sections)} document sections:")
#     print("\n".join(chosen_sections_indexes))
    
    header = """Answer the question using the provided context\n\nContext:\n"""
    
    return header + "".join(chosen_sections) + "\n\n Q: " + question + "\n A:"


In [63]:
COMPLETIONS_API_PARAMS = {
    # We use temperature of 0.0 because it gives the most predictable, factual answer.
    "temperature": 1,
    "max_tokens": 500,
    "model": COMPLETIONS_MODEL,
}

In [64]:
def answer_query_with_context(
    query: str,
    df: pd.DataFrame,
    document_embeddings: Dict[Tuple[str, str], np.array],
    show_prompt: bool = False
) -> str:
    prompt = construct_prompt(
        query,
        document_embeddings,
        df
    )
    
    if show_prompt:
        print(prompt)

    response = openai.Completion.create(
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n")

# Example Usage: 
Use the answer_query_with_context function to ask a question and get an answer based on the website's content.

In [65]:
answer_query_with_context("Quem é a Gisele?", df, document_embeddings, True)

Answer the question using the provided context

Context:

* Então… para quem não sabe, sou Gisele, 31 anos, casada, com uma filha que vai fazer 2 anos daqui a alguns dias. Também sou dona aqui do soumae.org, esse site existe desde 12 de setembro de 2014, onde o foco é dar dicas de decoração de festas infantis, chá de bebê, decoração de quartos e dicas para gestantes e mães… mas ele não é um blog pessoal… ele é meu trabalho. Gosto muito, lógico! Mas é trabalho!Eu não trabalho fora, decidimos, marido e eu, que eu ficaria em casa cuidando da nossa filhota, Isabela, e agradeço a Deus por ter essa possibilidade de passar esse tempo precioso com ela! Com isso acabei criando o soumae.org que hoje tem mais de 700 mil visualizações por mês! IURRU!Voltando ao assunto “blog novo”… As vezes eu até tento escrever algo um pouco mais pessoal no soumae… como dicas de atividades que faço com minha filha, ou outros assuntos (bem raro), mas esse não é o foco do soumae, até que por fim desisti de comparti

'Gisele é uma mulher de 31 anos, casada e mãe de uma filha que está para completar dois anos. Ela é a dona do site Soumae.org, onde ela dá dicas de decoração de festas infantis, chá de bebê, decoração de quartos e dicas para gestantes e mães. Ela também decidiu ficar em casa para cuidar de sua filha. Agora ela também está criando um blog pessoal para compartilhar junto com a sua família atividades que eles fazem como: contar histórias bíblicas para a filha da Gisele, planejar atividades semanais, receitas saudáveis, dicas de livros para casais, e filmes. Além disso, também irá compartilhar as mensagens de esperança que Gisele recebe através do The Verses Project.'

In [66]:
answer_query_with_context("Quem é a Ana Paula?", df, document_embeddings, True)

Answer the question using the provided context

Context:

* Hoje a entrevista é com a mamãe Ana Paula, integrante aqui do Sou Mãe. Ana tem uma baby linda, Beatriz de 8 meses!Vamos conhecer um pouquinho da Ana e saber os desafios de ser mãe!1. Qual foi a maior coisa que você teve que abrir mão nesses últimos anos?Do meu tempo... afinal filhos são um investimento de tempo integral.2. O que você faria se tivesse 3 horas a mais no seu dia?Dormiria... hahahahah3.  Ser mãe fez você ficar mais careta?Não, nem um pouco! Sinto que sou outra pessoa, me redescobrindo de novo.....4.  No momento o que você mais precisa? conversar ou ficar em silêncio?Conversar... filhos muito pequenos ainda, não conversam tanto....mas acho que logo, logo vou querer ficar em silêncio..hahahah5. O que mudou nas suas amizades desde que sua filha nasceu?Mudou bastante...as solteiras ou casadas sem filhos se distanciaram um pouco, mas em compensação fiz muitasssss amizades novas com mães, e é bom porque você pode ficar 

'Ana Paula é uma integrante do grupo Sou Mãe que tem uma filha de 8 meses, Beatriz. Ela compartilha sobre seus desafios de ser mãe no grupo.'

In [67]:
answer_query_with_context("O que motiva a Ana Paula?", df, document_embeddings, True)

Answer the question using the provided context

Context:

* Hoje a entrevista é com a mamãe Ana Paula, integrante aqui do Sou Mãe. Ana tem uma baby linda, Beatriz de 8 meses!Vamos conhecer um pouquinho da Ana e saber os desafios de ser mãe!1. Qual foi a maior coisa que você teve que abrir mão nesses últimos anos?Do meu tempo... afinal filhos são um investimento de tempo integral.2. O que você faria se tivesse 3 horas a mais no seu dia?Dormiria... hahahahah3.  Ser mãe fez você ficar mais careta?Não, nem um pouco! Sinto que sou outra pessoa, me redescobrindo de novo.....4.  No momento o que você mais precisa? conversar ou ficar em silêncio?Conversar... filhos muito pequenos ainda, não conversam tanto....mas acho que logo, logo vou querer ficar em silêncio..hahahah5. O que mudou nas suas amizades desde que sua filha nasceu?Mudou bastante...as solteiras ou casadas sem filhos se distanciaram um pouco, mas em compensação fiz muitasssss amizades novas com mães, e é bom porque você pode ficar 

'O que motiva a Ana Paula é saber que Deus confia nela, e que ela pode aprender a cuidar de seu bebê, e também tem motivação ao ver o marido dela se superar diariamente para cuidar dela, da casa e da bebê. Ela também é grata por ter uma família e amigos amorosos que a apoiam.'

In [68]:
answer_query_with_context("Me de 10 dicas de chá de bebê infantil?", df, document_embeddings, True)

Answer the question using the provided context

Context:

* Você pode ler agora alguns dos melhores artigos já publicados sobre chá de bebê aqui:Chá de bebê menino: 20 idéias de decoraçãoChá de bebê menina: 20 idéias de decoraçãoChá de bebê – SafariIdéias de lembrancinhas para chá de bebêChá de fraldas – como organizar a festa e fazer a lista de presentes9 dicas para fazer um chá de bebê baratoLembrancinhas de maternidadePasso a passo de convite para chá de bebê10 Brincadeiras para chá de bebê8 tipos de chá de bebê para você escolher18 Idéias de bolo de fraldasO que servir no chá de bebê?Livro de visitas - Guest BooksChá de Bebê com 12 ideias bem criativas9 dicas para fazer um chá de bebê baratoKit chá de bebê corujas - gratuito para downloadKit chá de bebê menino - gratuito para download35 modelos incríveis de bolo para chá de bebêChá de bebê – 36 ideias de bolosPara ver TODOS os posts sobre de chá de bebê clique: Chá de bebê
* Você está grávida e pensando nas muitas coisas que tem qu

'1. Escolha um tema colorido e divertido para o chá de bebê infantil; 2. Invite os amigos e familiares mais próximos; 3. Escolha um local apropriado; 4. Brinque com brincadeiras ou jogos relacionados ao tema; 5. Elabora uma lista completa com as coisas que você precisa comprar ou alugar; 6. Inclua algumas lembrancinhas para os convidados; 7. Procure por opções de decoração e itens de enxoval; 8. Escolha um bom bolo de bebê; 9. Encontre algumas frases e mensagens significativas relacionadas a bebê; 10. Solicita aplicativos que tornam mais fácil a organização do chá.'

# Practical Takeaways
* This approach allows you to leverage website content as a valuable resource for answering queries.
* By creating embeddings and calculating similarity, you can identify the most relevant sections of content for answering questions.
* Utilizing OpenAI's Completion API, you can generate accurate and context-aware answers based on the selected content.